-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

We are going to run all of the cells in the previous notebook to get our variables defined in this notebook.

In [3]:
%run "./00b Pandas Tutorial I"

## Missing Data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations. See the [Missing Data section](https://pandas.pydata.org/pandas-docs/version/0.23.0/missing_data.html#missing-data)

Reindexing allows you to change/add/delete the index on a specified axis. This returns a copy of the data.

In [5]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1],'E'] = 1
df1

To drop any rows that have missing data.

In [7]:
df1.dropna(how='any')

Filling missing data

In [9]:
df1.fillna(value=5)

To get the boolean mask where values are nan (note: had to change `isna` to `isnull` because our version of pandas is different).

In [11]:
pd.isnull(df1)

## Operations

See the Basic section on [Binary Ops](https://pandas.pydata.org/pandas-docs/version/0.23.0/basics.html#basics-binop)

Operations in general exclude missing data.

Performing a descriptive statistic

In [14]:
df.mean()

Same operation on the other axis

In [16]:
df.mean(1)

Operating with objects that have different dimensionality and need alignment. In addition, pandas automatically broadcasts along the specified dimension.

In [18]:
s = pd.Series([1,3,5,np.nan,6,8], index=dates).shift(2)
s

In [19]:
df.sub(s, axis='index')

Applying functions to the data

In [21]:
df.apply(np.cumsum)

In [22]:
df.apply(lambda x: x.max() - x.min())

Histogramming

See more at [Histogramming and Discretization](https://pandas.pydata.org/pandas-docs/version/0.23.0/basics.html#basics-discretization)

In [24]:
s = pd.Series(np.random.randint(0, 7, size=10))
s

In [25]:
s.value_counts()

String Methods

Series is equipped with a set of string processing methods in the str attribute that make it easy to operate on each element of the array, as in the code snippet below. Note that pattern-matching in str generally uses [regular expressions](https://docs.python.org/3/library/re.html) by default (and in some cases always uses them). See more at [Vectorized String Methods](https://pandas.pydata.org/pandas-docs/version/0.23.0/text.html#text-string-methods).

In [27]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s.str.lower()

## Merge

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

See the [Merging section](https://pandas.pydata.org/pandas-docs/version/0.23.0/merging.html#merging)

Concatenating pandas objects together with [concat()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.concat.html#pandas.concat)

In [30]:
df = pd.DataFrame(np.random.randn(10, 4))
df

In [31]:
# break it into pieces
pieces = [df[:3], df[3:7], df[7:]]
pd.concat(pieces)

Join

SQL style merges. See the [Database style joining](https://pandas.pydata.org/pandas-docs/version/0.23.0/merging.html#merging-join)

In [33]:
left = pd.DataFrame({'key': ['foo', 'foo'], 'lval': [1, 2]})
left

In [34]:
right = pd.DataFrame({'key': ['foo', 'foo'], 'rval': [4, 5]})
right

In [35]:
pd.merge(left, right, on='key')

Append rows to a dataframe. See the [Appending](https://pandas.pydata.org/pandas-docs/version/0.23.0/merging.html#merging-concatenation) section.

In [37]:
df = pd.DataFrame(np.random.randn(8, 4), columns=['A','B','C','D'])
df

In [38]:
s = df.iloc[3]
df.append(s, ignore_index=True)

## Grouping

By “group by” we are referring to a process involving one or more of the following steps

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure

See the [Grouping section](https://pandas.pydata.org/pandas-docs/version/0.23.0/groupby.html#groupby)

In [40]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})

df

Grouping and then applying the [sum()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.DataFrame.sum.html#pandas.DataFrame.sum) function to the resulting groups.

In [42]:
df.groupby('A').sum()

Grouping by multiple columns forms a hierarchical index, and again we can apply the sum function.

In [44]:
df.groupby(['A','B']).sum()

## Reshaping

See the sections on [Hierarchical Indexing](https://pandas.pydata.org/pandas-docs/version/0.23.0/advanced.html#advanced-hierarchical) and [Reshaping](https://pandas.pydata.org/pandas-docs/version/0.23.0/reshaping.html#reshaping-stacking).

In [46]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                     'one', 'two', 'one', 'two']]))

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])
df2 = df[:4]
df2

The [stack()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.DataFrame.stack.html#pandas.DataFrame.stack) method “compresses” a level in the DataFrame’s columns.

In [48]:
stacked = df2.stack()
stacked

With a “stacked” DataFrame or Series (having a MultiIndex as the index), the inverse operation of [stack()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.DataFrame.stack.html#pandas.DataFrame.stack) is [unstack()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.DataFrame.unstack.html#pandas.DataFrame.unstack), which by default unstacks the last level:

In [50]:
stacked.unstack()

In [51]:
stacked.unstack(1)

In [52]:
stacked.unstack(0)

See the section on [Pivot Tables](https://pandas.pydata.org/pandas-docs/version/0.23.0/reshaping.html#reshaping-pivot).

In [54]:
df = pd.DataFrame({'A' : ['one', 'one', 'two', 'three'] * 3,
                   'B' : ['A', 'B', 'C'] * 4,
                   'C' : ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
                   'D' : np.random.randn(12),
                   'E' : np.random.randn(12)})

df

We can produce pivot tables from this data very easily:

In [56]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

## Time Series

pandas has simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data). This is extremely common in, but not limited to, financial applications. See the [Time Series section](https://pandas.pydata.org/pandas-docs/version/0.23.0/timeseries.html#timeseries).

In [58]:
rng = pd.date_range('1/1/2012', periods=100, freq='S')
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
ts.resample('5Min').sum()

Time zone representation:

In [60]:
rng = pd.date_range('3/6/2012 00:00', periods=5, freq='D')
ts = pd.Series(np.random.randn(len(rng)), rng)
ts

In [61]:
ts_utc = ts.tz_localize('UTC')
ts_utc

Converting to another time zone:

In [63]:
ts_utc.tz_convert('US/Eastern')

Converting between time span representations:

In [65]:
rng = pd.date_range('1/1/2012', periods=5, freq='M')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

In [66]:
ps = ts.to_period()
ps

In [67]:
ps.to_timestamp()

Converting between period and timestamp enables some convenient arithmetic functions to be used. In the following example, we convert a quarterly frequency with year ending in November to 9am of the end of the month following the quarter end:

In [69]:
prng = pd.period_range('1990Q1', '2000Q4', freq='Q-NOV')
ts = pd.Series(np.random.randn(len(prng)), prng)
ts.index = (prng.asfreq('M', 'e') + 1).asfreq('H', 's') + 9
ts.head()

## Categoricals

pandas can include categorical data in a DataFrame. For full docs, see the [categorical introduction](https://pandas.pydata.org/pandas-docs/version/0.23.0/categorical.html#categorical) and the [API documentation](https://pandas.pydata.org/pandas-docs/version/0.23.0/api.html#api-categorical).

In [71]:
df = pd.DataFrame({"id":[1,2,3,4,5,6], "raw_grade":['a', 'b', 'b', 'a', 'a', 'e']})

Convert the raw grades to a categorical data type.

In [73]:
df["grade"] = df["raw_grade"].astype("category")
df["grade"]

Rename the categories to more meaningful names (assigning to Series.cat.categories is inplace!).

In [75]:
df["grade"].cat.categories = ["very good", "good", "very bad"]

Reorder the categories and simultaneously add the missing categories (methods under Series .cat return a new Series by default).

In [77]:
df["grade"] = df["grade"].cat.set_categories(["very bad", "bad", "medium", "good", "very good"])
df["grade"]

Sorting is per order in the categories, not lexical order.

In [79]:
df.sort_values(by="grade")

Grouping by a categorical column also shows empty categories.

In [81]:
df.groupby("grade").size()

## Plotting

See the [Plotting](https://pandas.pydata.org/pandas-docs/version/0.23.0/visualization.html#visualization) docs.

In [83]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts = ts.cumsum()
fig = ts.plot()
display(fig.figure) # Need to wrap in display for Databricks

On a DataFrame, the [plot()](https://pandas.pydata.org/pandas-docs/version/0.23.0/generated/pandas.DataFrame.plot.html#pandas.DataFrame.plot) method is a convenience to plot all of the columns with labels:

In [85]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')
display(plt.show())

## Getting Data In/Out

[Writing to a csv file.](https://pandas.pydata.org/pandas-docs/version/0.23.0/io.html#io-store-in-csv)

In [88]:
df.to_csv('foo.csv')

[Reading from a csv file.](https://pandas.pydata.org/pandas-docs/version/0.23.0/io.html#io-read-csv-table)

In [90]:
pd.read_csv('foo.csv')

HDF5

Reading and writing to [HDFStores](https://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5).

Writing to a HDF5 Store.

In [92]:
%sh /databricks/python/bin/pip install tables
/databricks/python/bin/pip install xlrd

In [93]:
df.to_hdf('foo.h5','df')

Reading from a HDF5 Store.

In [95]:
pd.read_hdf('foo.h5','df')

Excel

Reading and writing to [MS Excel](https://pandas.pydata.org/pandas-docs/version/0.23.0/io.html#io-excel).

Writing to an excel file.

In [97]:
df.to_excel('foo.xlsx', sheet_name='Sheet1')

Reading from an excel file.

In [99]:
pd.read_excel('foo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])

## Gotchas

If you are attempting to perform an operation you might see an exception like:

In [101]:
if pd.Series([False, True, False]):
    print("I was true")

See [Comparisons](https://pandas.pydata.org/pandas-docs/version/0.23.0/basics.html#basics-compare) for an explanation and what to do.

See [Gotchas](https://pandas.pydata.org/pandas-docs/version/0.23.0/gotchas.html#gotchas) as well.

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>